## Example 14 - Pick up variable load

In [1]:
# Package(s) related to time, space and id
import datetime, time
import platform
import random
import os

# You need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# Spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# Package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Tranport network analysis package
import transport_network_analysis.core as core
import transport_network_analysis.graph_module as graph_module
import transport_network_analysis.model as model

# Used for mathematical functions
import math             

# Used for making the graph to visualize our problem
import networkx as nx  

### Import data

In [40]:
# Import stations and lines from excel sheets
df1 = pd.read_excel("../../Data.xlsx", sheet_name = "Stations")
df2 = pd.read_excel("../../Data.xlsx", sheet_name = "Lijnen")
metro52 = df1[df1["Line"] == "metro52"]
tram24 = df1[df1["Line"] == "tram24"]

df_list = [metro52, tram24]

from ast import literal_eval
df2.loc[:,'Route1'] = df2.loc[:,'Route1'].apply(lambda x: literal_eval(x))
df2.loc[:,'Route2'] = df2.loc[:,'Route2'].apply(lambda x: literal_eval(x))

In [39]:
df2

,Line,Route1,Route2,Duration,Capacity
0,metro52,"[Noord, Noorderpark, CentraalStation, Rokin, V...","['Zuid', 'Europaplein', 'DePijp', 'Vijzelgrach...","[2, 2, 2, 2, 2, 3]",100
1,tram24,"[CentraalStation, Dam, Rokin, Muntplein, Vijze...","['DeBoelelaanVU', 'VUMedischCentrum', 'Amstelv...","[3, 2, 1, 3, 1, 2, 3, 2, 1 ,1, 2, 2, 1, 1, 1, ...",100


In [46]:
# Create dictionary for all transport lines
lines = {}
durations = []

for index, row in df2.iterrows():
    lines[row["Line"]] = (row["Route1"], row["Route2"])
    
edges = []
for i in lines.values():
    for x in range(len(i[0])):
        try:
            edges.append((i[0][x], i[0][x + 1]), )
            edges.append((i[0][x + 1], i[0][x]), )
        except:
            break
            
print(edges)

[('Noord', 'Noorderpark'), ('Noorderpark', 'Noord'), ('Noorderpark', 'CentraalStation'), ('CentraalStation', 'Noorderpark'), ('CentraalStation', 'Rokin'), ('Rokin', 'CentraalStation'), ('Rokin', 'Vijzelgracht'), ('Vijzelgracht', 'Rokin'), ('Vijzelgracht', 'DePijp'), ('DePijp', 'Vijzelgracht'), ('DePijp', 'Europaplein'), ('Europaplein', 'DePijp'), ('Europaplein', 'Zuid'), ('Zuid', 'Europaplein'), ('CentraalStation', 'Dam'), ('Dam', 'CentraalStation'), ('Dam', 'Rokin'), ('Rokin', 'Dam'), ('Rokin', 'Muntplein'), ('Muntplein', 'Rokin'), ('Muntplein', 'Vijzelgracht'), ('Vijzelgracht', 'Muntplein'), ('Vijzelgracht', 'MarieHeinekenplein'), ('MarieHeinekenplein', 'Vijzelgracht'), ('MarieHeinekenplein', 'DePijp'), ('DePijp', 'MarieHeinekenplein'), ('DePijp', 'RoelofHartplein'), ('RoelofHartplein', 'DePijp'), ('RoelofHartplein', 'GerritvdVeenstraat'), ('GerritvdVeenstraat', 'RoelofHartplein'), ('GerritvdVeenstraat', 'Beethovenstraat'), ('Beethovenstraat', 'GerritvdVeenstraat'), ('Beethovenstraat

### Create graph

The cel below visualizes the problem. In graph theory the red dots are called *edges* and the lines are called *vertices*. Vessels (or any other object) move from edge 1 to edge 3 and from edge 4 to edge 2. The added complexity is that vertice 5-6 only allows traffic in one direction at a time. Vessels can travel simultanously in one direction.

In [5]:
Node = type('Station', (core.Identifiable, core.Log, core.Locatable, core.HasResource, core.Station), {})
nodes = []

In [12]:
# Create graph
FG = nx.DiGraph()

for df in df_list:
    # Create all stations
    stations = []
    
    for index, row in df.iterrows():
        stations.append({"env": [],
                         "name": row["HalteNaam"],
                         "geometry": shapely.geometry.Point(row["HalteLat"], row["HalteLon"]),
                         "capacity": row["Capacity"]}) 
        
        
    
    #Create all nodes and edges
    line_nodes = []
    
    for station in stations:
        node = Node(**station)
        nodes.append(node)
        line_nodes.append(node)
        
    # Create positions and add nodes to graph with right position
    positions = {}
    
    for node in nodes:
        positions[node.name] = (node.geometry.y, node.geometry.x)
        FG.add_node(node.name, geometry = node.geometry, object_type = node)
    

    
    for edge in edges:
        FG.add_edge(edge[0].name, edge[1].name, duration = dur[(edge[0], 
                                                                edge[1])][0], line = dur[(edge[0], edge[1])][1])


In [8]:
# Create graph with edges

# Create positions and add nodes to graph with right position
positions = {}
for node in nodes:
    positions[node.name] = (node.geometry.y, node.geometry.x)
    FG.add_node(node.name, geometry = node.geometry, object_type = node)

# for edge in edges:
#     FG.add_edge(edge[0].name, edge[1].name, duration = dur[(edge[0], edge[1])][0], line = dur[(edge[0], edge[1])][1])

# nx.draw(FG, positions, node_size=10)
# plt.show()

# edges = {(node_1, node_2): (2, "metro52"), 
#         (node_2, node_1): (2, "metro52"), 
#         (node_2, node_3): (2, "metro52"), 
#         (node_3, node_2): (2, "metro52"), 
#         (node_3, node_4): (2, "metro52"), 
#         (node_4, node_3): (2, "metro52"),
#         (node_4, node_5): (2, "metro52"), 
#         (node_5, node_4): (2, "metro52"), 
#         (node_5, node_6): (2, "metro52"), 
#         (node_6, node_5): (2, "metro52"), 
#         (node_6, node_7): (2, "metro52"), 
#         (node_7, node_6): (2, "metro52"),
#         (node_7, node_8): (3, "metro52"), 
#         (node_8, node_7): (3, "metro52"),
#         (node_9, node_10): (3, "tram24"), 
#         (node_10, node_9): (3, "tram24"), 
#         (node_10, node_11): (2, "tram24"), 
#         (node_11, node_10): (2, "tram24"), 
#         (node_11, node_12): (1, "tram24"), 
#         (node_12, node_11): (1, "tram24"),
#         (node_12, node_13): (3, "tram24"), 
#         (node_13, node_12): (3, "tram24"), 
#         (node_13, node_14): (1, "tram24"), 
#         (node_14, node_13): (1, "tram24"), 
#         (node_14, node_15): (2, "tram24"), 
#         (node_15, node_14): (2, "tram24"),
#         (node_15, node_16): (3, "tram24"), 
#         (node_16, node_15): (3, "tram24"),
#         (node_16, node_17): (2, "tram24"), 
#         (node_17, node_16): (2, "tram24"), 
#         (node_17, node_18): (1, "tram24"), 
#         (node_18, node_17): (1, "tram24"), 
#         (node_18, node_19): (1, "tram24"), 
#         (node_19, node_18): (1, "tram24"),
#         (node_19, node_20): (2, "tram24"), 
#         (node_20, node_19): (2, "tram24"), 
#         (node_20, node_21): (2, "tram24"), 
#         (node_21, node_20): (2, "tram24"), 
#         (node_21, node_22): (1, "tram24"), 
#         (node_22, node_21): (1, "tram24"),
#         (node_22, node_23): (1, "tram24"), 
#         (node_23, node_22): (1, "tram24"),
#         (node_23, node_24): (1, "tram24"), 
#         (node_24, node_23): (1, "tram24"), 
#         (node_24, node_25): (2, "tram24"), 
#         (node_25, node_24): (2, "tram24"), 
#         (node_25, node_26): (2, "tram24"), 
#         (node_26, node_25): (2, "tram24")}

### Create moving parts

In [23]:
def generate_passenger(origin, destination, environment):
    """ Generate a passenger """
    Passenger = type('Passenger', (core.Identifiable, core.Routeable, core.Log), {})
    
    # Name of the object
    name = "Passenger"
    
    # Geometry of the origin
    geometry = nx.get_node_attributes(environment.FG, "object_type")[origin].geometry
    
    # Travel direction
    route = nx.dijkstra_path(environment.FG, origin, destination, weight = 'duration')
    
    return Passenger(**{"env": environment, 
                        "name": name, 
                        "route": route})

In [24]:
def generate_metro(line, environment):
    """ Generate a metro """
    Metro = type('Metro', (core.Identifiable, core.Movable, core.HasContainer, core.HasResource, 
                           core.Routeable, core.Mover), {})

    # Name of the object
    name = "Metro"

    # Travel direction
#     route = nx.dijkstra_path(environment.FG, origin, destination, weight = 'duration')
    route = line    
    # Geometry of origin

    geometry = nx.get_node_attributes(environment.FG, "object_type")[route[0]].geometry
    
    # Maximum passengers
    capacity = 50
    
    return Metro(**{"env": environment, 
                    "name": name, 
                    "route": route, 
                    "capacity": capacity, 
                    "geometry": geometry})

### Spawn the metro trains

In [25]:
def vehicle(env, start, frequency, line):
    yield env.timeout(start * 60 * 60)
    while True:
        metro = generate_metro(line, env)
        env.metros.append(metro)
        env.process(metro.move())
        
        # One metro every X minutes
        yield env.timeout(frequency * 60)

In [26]:
def passengers(env):
    yield env.timeout(5.5 * 60 * 60)
    while True:
        origin, destination = random.sample(env.FG.nodes, 2)
        
        passenger = generate_passenger(origin, destination, env)
        FG.nodes[origin]["object_type"].units.append(passenger)
        
        passenger.log_entry("Waiting for metro start", env.now, 0, nx.get_node_attributes(env.FG, "object_type")[origin].geometry)
        env.passengers.append(passenger)
        
        # Random arrivals of passengers with mean interarrival time of 30 seconds
        yield env.timeout(random.random() * 60)

### Run simulation

In [27]:
# Simulation start time
simulation_start = datetime.datetime(2019, 1, 1)

# Make simulation environment
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.FG = FG

# For loggin
env.metros = []
env.passengers = []

# Start the simulation
env.process(vehicle(env, 5.5, 6, lines[52][0]))
env.process(vehicle(env, 5.585, 6, lines[52][1]))
env.process(vehicle(env, 6, 10, lines[24][0]))
env.process(vehicle(env, 6, 10, lines[24][1]))
env.process(passengers(env))
env.run(until = env.now + 1 * 24 * 60 * 60)

### Check results

In [28]:
env.metros[54].route

['DeBoelelaanVU',
 'VUMedischCentrum',
 'Amstelveenseweg',
 'IJsbaanpad',
 'OlympischStadion',
 'Olympiaweg',
 'Olympiaplein',
 'Minervaplein',
 'Beethovenstraat',
 'GerritvdVeenstraat',
 'RoelofHartplein',
 'DePijp',
 'MarieHeinekenplein',
 'Vijzelgracht',
 'Muntplein',
 'Rokin',
 'Dam',
 'CentraalStation']

In [40]:
env.passengers[0].log

{'Message': ['Waiting for metro start',
  'Waiting for metro stop',
  'In metro start',
  'In metro stop'],
 'Timestamp': [datetime.datetime(2019, 1, 1, 5, 30),
  datetime.datetime(2019, 1, 1, 6, 8, 15),
  datetime.datetime(2019, 1, 1, 6, 8, 15),
  datetime.datetime(2019, 1, 1, 6, 10, 15)],
 'Value': [0, 0, 0, 0],
 'Geometry': [<shapely.geometry.point.Point at 0xa131a9b00>,
  <shapely.geometry.point.Point at 0x10b263cc0>]}

In [45]:
# What routes did people (i = person) take that took this metro
for i in env.metros[9].units:
    print(i.route)

In [51]:
env.metros[10].log

{'Message': ['Driving from CentraalStation to Dam start',
  'Driving from CentraalStation to Dam stop',
  'Unloading start',
  'Unloading stop',
  'Loading start',
  'Loading stop',
  'Driving from Dam to Rokin start',
  'Driving from Dam to Rokin stop',
  'Unloading start',
  'Unloading stop',
  'Driving from Rokin to Muntplein start',
  'Driving from Rokin to Muntplein stop',
  'Unloading start',
  'Unloading stop',
  'Loading start',
  'Loading stop',
  'Driving from Muntplein to Vijzelgracht start',
  'Driving from Muntplein to Vijzelgracht stop',
  'Unloading start',
  'Unloading stop',
  'Loading start',
  'Loading stop',
  'Driving from Vijzelgracht to MarieHeinekenplein start',
  'Driving from Vijzelgracht to MarieHeinekenplein stop',
  'Unloading start',
  'Unloading stop',
  'Loading start',
  'Loading stop',
  'Driving from MarieHeinekenplein to DePijp start',
  'Driving from MarieHeinekenplein to DePijp stop',
  'Unloading start',
  'Unloading stop',
  'Loading start',
  'L

In [53]:
env.passengers[1].__dict__

{'env': <simpy.core.Environment at 0xa1e902e80>,
 'log': {'Message': ['Waiting for metro start'],
  'Timestamp': [datetime.datetime(2019, 1, 1, 5, 30, 33, 480960)],
  'Value': [0],
  'Geometry': [<shapely.geometry.point.Point at 0xa131b4f98>]},
 'route': ['Zuid',
  'Europaplein',
  'DePijp',
  'RoelofHartplein',
  'GerritvdVeenstraat'],
 'complete_path': None,
 'name': 'Passenger',
 'id': '4668971e-6bf2-11e9-a5ae-7831c1d08f6a'}

In [19]:
pd.DataFrame.from_dict(env.metros[1].log)

,Message,Timestamp,Value,Geometry
0,Loading start,2019-01-01 05:35:06,0,POINT (52.33908015 4.87509853)
1,Loading stop,2019-01-01 05:35:06,0,POINT (52.33908015 4.87509853)
2,Driving from Zuid to Europaplein start,2019-01-01 05:35:06,0,POINT (52.33908015 4.87509853)
3,Driving from Zuid to Europaplein stop,2019-01-01 05:38:06,0,POINT (52.34185482 4.8913843)
4,Unloading start,2019-01-01 05:38:06,0,POINT (52.34185482 4.8913843)
5,Unloading stop,2019-01-01 05:38:06,0,POINT (52.34185482 4.8913843)
6,Loading start,2019-01-01 05:38:21,0,POINT (52.34185482 4.8913843)
7,Loading stop,2019-01-01 05:38:21,0,POINT (52.34185482 4.8913843)
8,Driving from Europaplein to DePijp start,2019-01-01 05:38:21,0,POINT (52.34185482 4.8913843)
9,Driving from Europaplein to DePijp stop,2019-01-01 05:40:21,0,POINT (52.35296212 4.89101069)
